PyCity School Analysis
February 8, 2019
Scott McEachern

In [1]:
#- Load Datasets into DataFrame
import os
import pandas as pd


# Schools DataFrame
schoolPath = os.path.join(".", "Resources", "schools_complete.csv")

school_df = pd.read_csv(schoolPath)


# Student DataFrame
studentPath = os.path.join(".", "Resources", "students_complete.csv")

student_df = pd.read_csv(studentPath)

In [11]:
#-District Summary


#- Calculate Total Schools
totalNumSchools = school_df.shape[0]


#- Calculate Total Students
totalNumStudents = student_df.shape[0]
totalNumStudents = f"{'{:,}'.format(totalNumStudents)}"




# Result Dataframe
districtSummaryResults = {
    'Total Schools': [totalNumSchools],
    'Total Students': [totalNumStudents]
                }

districtSummary_df = pd.DataFrame(districtSummaryResults)

print(totalNumStudents)
print(districtSummary_df)

39,170
   Total Schools Total Students
0             15         39,170
